In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [4]:
data=pd.read_csv(r"D:\deepLearningNLP\ANNClassification\Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
##Preprocess the data
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [6]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
#Encode categorical variables
labelEncoderGender=LabelEncoder()
data['Gender']=labelEncoderGender.fit_transform(data['Gender'])

In [8]:
##Onehot encode 'Geography' column
onehotEncoderGeo=OneHotEncoder(handle_unknown='ignore')
geoEncoded=onehotEncoderGeo.fit_transform(data[['Geography']]).toarray()
geoEncoded



array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [9]:
geoEncodeddf=pd.DataFrame(geoEncoded,columns=onehotEncoderGeo.get_feature_names_out(['Geography']))
geoEncodeddf

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [10]:
#Combine onehot encoded values with original data
data=pd.concat([data.drop('Geography',axis=1),geoEncodeddf],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [11]:
#Split the data into features and target
X=data.drop('EstimatedSalary',axis=1) #independent variables
y=data['EstimatedSalary'] #dependent variable

In [12]:
#Split the data in training and testing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [13]:
#Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [14]:
#Save the encoders and scaler for later use
with open('labelEncoderGenderRegression.pkl','wb') as file:
    pickle.dump(labelEncoderGender,file)
with open('onehotEncoderGeoRegression.pkl','wb') as file:
    pickle.dump(onehotEncoderGeo,file)
with open('scalerRegression.pkl','wb')as file:
    pickle.dump(scaler,file)

In [15]:
## ANN Regression Problem Statement
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [16]:
#Build the model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1)#Output layer for regression
])

##Compile the model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
##Setup the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

log_dir="logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [18]:
##Setup Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [19]:
## Train the model
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 3s 5ms/step - loss: 100375.0391 - mae: 100375.0391 - val_loss: 98511.2109 - val_mae: 98511.2109
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 99606.3125 - mae: 99606.3125 - val_loss: 96956.9688 - val_mae: 96956.9688
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 96894.3203 - mae: 96894.3203 - val_loss: 92986.3125 - val_mae: 92986.3125
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 91553.2109 - mae: 91553.2109 - val_loss: 86351.6641 - val_mae: 86351.6641
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 83738.5078 - mae: 83738.5078 - val_loss: 77792.0859 - val_mae: 77792.0859
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 74629.0547 - mae: 74629.0547 - val_loss: 68921.6484 - val_mae: 68921.6484
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 65851.5703 - mae: 65851.570

In [20]:
model.save('modelRegression.h5')

d:\deepLearningNLP\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [21]:
##Load Tensorboard Extension
%load_ext tensorboard

In [22]:
%tensorboard --logdir logs/fit20251204-142849

In [23]:
##Evaluate model on the test data
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Test Mean Absolute Error :{test_mae}')

63/63 [==============================] - 0s 2ms/step - loss: 50235.7773 - mae: 50235.7773
Test Mean Absolute Error :50235.77734375


In [24]:
model.save('regressionModel.h5')